## TDI Project Constants and Functions

In [2]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
import seaborn as sns
sns.set(style='white', font_scale=1)
import matplotlib.patches as mpatches

from datetime import datetime

import csv

from matplotlib import rcParams

import math
from difflib import SequenceMatcher
from ipywidgets import interact, interactive, fixed, interact_manual
from textwrap import wrap

import sklearn
from sklearn.utils import check_random_state
from sklearn.cluster import KMeans

In [12]:
focus_industry = ['Business Services',
 'Communications',
 'Miscellaneous Retail',
 'Food and Kindred Products',
 'Eating and Drinking Places',
 'Apparel, Finished Products from Fabrics & Similar Materials',
 'Apparel and Accessory Stores',
 'Insurance Carriers',
 'Food Stores',
 'Wholesale Trade - Nondurable Goods',
 'Transportation Services',
 'Real Estate',
 'Hotels, Rooming Houses, Camps, and Other Lodging Places',
 'Transportation by Air',
 'Motion Pictures',
 'Health Services',
 'Amusement and Recreation Services']

num_cols = ['checkins', 'were_here_count', 'likes', 'talking_about_count']

notime_cols = ['dataset_id', 'facebook_id', 'username','user',
       'conm', 'gvkey', 'cusip', 'sic', 
       'diff1','diff2','match_status',
       'sic_2digit', 'industry_short']

period = {'Day': 'D',
          'Week': 'W',
          'Quarter': 'Q',
          'Month': 'M',
          'Year': 'A'}

interactions = {'Were-Here': 'were_here_count',
            'Check-Ins': 'checkins',
            'Likes': 'likes',
            'Talking-About':'talking_about_count'}

grouping = {'Industry': 'industry_short',
            'Cluster': 'cluster',
            'Form': 'virtual_c'
            }

## Data Preparation

In [3]:
def get_time_period(df, p):
    '''
    Convert datatime to specific time interval.
    
    df: A Pandas DataFrame.
    p: A string, indicates time interval.
    
    Return a Pandas DataFrame.
    '''
    
    if p == 'D':
        df[p] = df['time_2'].dt.date
    else:    
        df[p] = df['time_2'].dt.to_period(p)
    
    return df

## Trend of Company Social Media Performance

In [2]:
def industry_baseline(data, industry, col, p):
    '''
    Compute Industry-level metrics.
    
    data: A Pandas DataFrame.
    industry: A string, industry name.
    col: A string, column name.
    p: A string, indicates time interval.
    
    Return a tuple of two Pandas DataFrame.
    
    '''
    
    if industry == 'Not Available':
        return None
    
    good = data['industry_short'] == industry
    df_ind = data[good]
    
    df_ind = get_time_period(df_ind,p)
    
    # calculate the grand mean
    df0 = df_ind.groupby(['time_period','user'])[col].mean()
    df = df0.reset_index().groupby(['time_period'])[col].agg({'mean','median','max','min','std','count'}).reset_index()

    return df,df0

def group_baseline(data, group, group_col, col, p):
    '''
    Compute Industry-level metrics.
    
    data: A Pandas DataFrame.
    group: A string or an int, group label.
    group_col: A string, name of grouping column.
    col: A string, column name.
    p: A string, indicates time interval.
    
    Return a tuple of two Pandas DataFrame.
    
    '''
    
    if group == 'Not Available':
        return None
    
    good = data[group_col] == group
    df_ind = data[good]
    
    #df_ind = get_time_period(df_ind,p)
    df_ind['time_period'] = df_ind[p]
    
    # calculate the grand mean
    df0 = df_ind.groupby(['time_period','user'])[col].mean()
    df = df0.reset_index().groupby(['time_period'])[col].agg({'mean','median','max','min','std','count'}).reset_index()

    return df,df0

def company_trend(data, company, col, p):
    '''
    Compute firm-level trend.
    
    data: A Pandas DataFrame.
    company: A string, company name.
    col: A string, column name to the metric of interest.
    p: A string, indicates time interval.
    
    Return a Pandas DataFrame. 
    
    '''
    
    good = data['user'] == company
    df = data[good]
    
    #df = get_time_period(df, p)
    df['time_period'] = df[p]
    
    grouped = df.sort_values('time_period').groupby(['time_period'])
    
    df_byperiod = grouped[col].agg({'mean'}).reset_index()
    
    df_byperiod[col+'_inc'] = df_byperiod['mean'].pct_change()
    df_byperiod = df_byperiod.rename(columns = {'mean': 'avg'})
    
    return df_byperiod


def social_trend(company, interaction, by,
                 data,
                 compare = True, 
                 compare_by = 'Industry',
                 percentage_change = False):
    '''
    A wrap up function for data visualization.
    
    company: A string, company name.
    interaction: A string, specify which type of online activities to look at.
    by: A string, indicates time interval.
    data: A Pandas DataFrame. 
    compare: A Bool, whether to compare the company with its identified industry. Default True.
    compare_by: A stirng, which group to compare from, default 'Industry'.
    percentage_change: A Bool, whether to show percentage change of certain trend. Default False.
    
    Return None.
    '''

    col = interactions.get(interaction)
    p = period.get(by)
    grp_col = grouping.get(compare_by)
    group = 'Not Available'
    
    if grp_col not in data.columns.tolist():
        compare = False
    else:
        group = data[data['user'] == company][grp_col].values[0]
    
    df_plot = company_trend(data,company,col,p)
    df_plot['t'] = df_plot.index
    
    fig, ax = plt.subplots(figsize = (15,6))

    if compare and group != 'Not Available':
        
        df_ind,_ = group_baseline(data,group,grp_col,col,p)
        
        df_plot = df_plot.merge(df_ind,how = 'left',on='time_period')

        ax.plot(df_plot['t'],df_plot['mean'],lw=2, label='{1} Baseline: {0}'.format(group,compare_by),color='yellow')
        ax.plot(df_plot['t'],df_plot['avg'],lw=2.5, label=company, color='black')
        ax.fill_between(df_plot['t'],df_plot['mean']-df_plot['std'],df_plot['mean']+df_plot['std'], 
                        facecolor='yellow', label='{} Mean +/- sd'.format(compare_by), alpha=0.2)
        ax.fill_between(df_plot['t'],df_plot['min'],df_plot['max'], 
                        facecolor='blue',label='{} Minimum and Maximum'.format(compare_by), alpha=0.1)
        ax.legend(loc='upper left')
        ax.set_title("Trend of {0}: {1} ({3}: {2})".format(interaction,company,group,compare_by))
        ax.set_ylabel("Counts")
    
    elif percentage_change:

        baseline = 0
        temp = df_plot[col+'_inc'].values.tolist()
        temp.pop(0)

        ax.bar(range(len(temp)), temp, bottom=baseline)

        ax.set_title("Percentage Change of {0}: {1} ({3}: {2})".format(interaction,company,group,compare_by))
        ax.set_ylabel("Percentage")
        
    else:
        ax.plot(df_plot['t'],df_plot['avg'],lw=2.5, label=company, color='black')
        ax.set_title("Trend of {0}: {1} ({3}: {2})".format(interaction,company,group,compare_by))
        ax.set_ylabel("Counts")

    ax.set_xlabel(by)
    freq = len(df_plot['t'].values.tolist())//20 + 1
    ax.set_xticks(df_plot['t'].values.tolist()[::freq])
    ax.set_xticklabels(df_plot['time_period'].sort_values('time_2', ascending = False).values.tolist()[::freq], rotation=45)
    sns.despine()

## Looking at Volatility and Growth of Social Media Performance

In [7]:
def company_summary(df,p):
    
    '''
    Compute firm level statistics
    
    df: A Pandas DataFrame.
    p: A string, indicates time interval.
    
    Return a tuple of two Pandas DataFrames. 
    '''
    
    #df = get_time_period(df, p)
    df['time_period'] = df[p]

    # group by user and time period
    df_bytime = df.groupby(['user','time_period'])[num_cols].mean()
    
    # calculate grand mean and other statistics
    df_stats = df_bytime.reset_index().groupby(['user'])[num_cols].agg({'mean','max','std','count'}).reset_index()
    df_stats.columns = df_stats.columns.map('{0[0]}_{0[1]}'.format) 
    df_stats = df_stats.rename(columns = {'user_':'user'})
    
    # volatility: (Coefficient of Variance)COV
    df_cov = df_bytime.reset_index().groupby(['user'])[num_cols].apply(lambda x: x.std()/x.mean()).reset_index()
    df_cov = df_cov.rename(columns = {'checkins': 'checkins_cov',
                                      'were_here_count':'were_here_count_cov',
                                      'likes':'likes_cov',
                                      'talking_about_count':'talking_about_count_cov'})
    # merge two datasets
    df_merge = df_cov.merge(df_stats)
    
    # percentage change: total percentage change 
    df_percent = df_bytime.reset_index()
    colnames = [col+'_inc' for col in num_cols]
    df_percent[colnames] = df_percent.sort_values('time_period').groupby(['user'])[num_cols].pct_change()
    df_percent_sum = df_percent.groupby(['user'])[colnames].sum().reset_index()
    
    # merge two datasets
    df_merge = df_merge.merge(df_percent_sum)
    
    return df_merge, df_bytime

## clustering 

In [8]:
def cluster(data, random_state, n_clusters=4):
    '''
    Fits and predicts k-means clustering on "array"
    
    Parameters
    ----------
    data: A Pandas DataFrame
    random_state: Random seed, e.g. check_random_state(0)
    n_clusters: The number of clusters. Default: 4
    
    Returns
    -------
    A tuple (sklearn.KMeans, np.ndarray)
    '''
    
    # build the model
    model = KMeans(n_clusters = n_clusters, random_state = random_state)
    
    # fit our data
    model.fit(data)
    
    # obtain the predictions
    clusters = model.predict(data)
    
    return model, clusters

def plot_inertia(data, start=1, end=10):
    '''
    Increase the number of clusters from "start" to "end" (inclusive).
    Finds the inertia of k-means clustering for different k.
    Plots inertia as a function of the number of clusters.

    
    Parameters
    ----------
    data: A Pandas DataFrame
    start: An int. Default: 1
    end: An int. Default: 10
    
    Returns
    -------
    A matplotlib.Axes instance.
    '''
    
    # get the numbers of cluters 
    n_cl = list(range(start,end+1))
    
    # iniate an zeros for intertia values
    inertia = np.zeros(len(n_cl))
    
    # compute interia value for each model
    for idx, n in enumerate(n_cl):
        # get the model
        k_means, clusters = cluster(data, random_state = check_random_state(0), n_clusters = n)
        inertia[idx] = k_means.inertia_
        
    # make the plot
    
    sns.set(style="white")
    fig, ax = plt.subplots(figsize=(10,6))
    
    # draw the points 
    ax.set_xlim(1,10)
    plt.plot(n_cl, inertia)
    
    # add decorations
    ax.set_title("The elbow method")
    ax.set_xlabel("Number of clusters")
    ax.set_ylabel("Inertia")
    sns.set(font_scale=1.5)

    return ax

def plot_pair(data, clusters, cols):
    '''
    Uses seaborn.PairGrid to visualize the data distribution
    when axes are the first four principal components.
    Diagonal plots are histograms. The off-diagonal plots are scatter plots.
    
    Parameters
    ----------
    data: A numpy array. 
    
    Returns
    -------
    A seaborn.axisgrid.PairGrid instance.
    '''
    
    data = data[cols]
    data['clusters'] = clusters

    # make pair plot
    #fig, ax = plt.subplots(figsize=(20,20))
    sns.set(style="white")
    ax = sns.PairGrid(data, vars = cols, hue ="clusters")
    
    # plots on the diagonal are histograms 
    ax.map_diag(plt.hist)
    
    # plots off-diagonal are scatter plot
    ax.map_offdiag(plt.scatter)
    
    return ax

In [11]:
#! cat workflow.sh